#### Load Libraries and Demo Codes

In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

import julia; julia.install(quiet=True)
from julia import Main

import sympy as sp
import numpy as np
from numpy.linalg import qr, svd

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import param
import holoviews as hv
import panel as pn

pn.extension('katex', 'plotly')
hv.extension('plotly', 'bokeh', logo=False)

%load_ext julia.magic

In [14]:
%%html
<script src="https://cdnjs.cloudflare.com/ajax/libs/paper.js/0.12.18/paper-full.min.js"></script>

In [2]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, RowEchelon, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `~/NOTEBOOKS/elementary-linear-algebra/GenLinAlgProblems`


In [3]:
#%output backend="plotly"
def spherical_to_cartesian(r, theta_deg, phi_deg):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    return r * np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])
def rotation_matrix(axis, theta):
    """
    Returns the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d),     2*(b*d + a*c)],
        [2*(b*c + a*d),     a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c),     2*(c*d + a*b),     a*a + d*d - b*b - c*c]
    ])
def project_to_plane(v, normal):
    n = normal / np.linalg.norm(normal)
    return v - np.dot(v, n) * n

In [4]:
#%output backend="plotly"
def dynamic_name(group, name):
    """
    Generates a standardized dynamic trace name.

    Args:
        group (str): Type of the trace ("plane", "vector", "arc", etc.)
        name (str): Descriptive name of the trace.

    Returns:
        str: Combined dynamic trace name.
    """
    return f"dynamic:{group}:{name}"
def replace_traces(fig, new_traces, group="dynamic"):
    """
    Safely replaces all traces in the figure that belong to the given dynamic group.
    """
    prefix   = group if group.endswith(":") else f"{group}:"
    fig.data = [trace for trace in fig.data if not (hasattr(trace, "name") and trace.name.startswith("dynamic:"))]
    fig.add_traces(new_traces)
def create_plane(origin, v1, v2, size=2.0):
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    points = [origin + a * v1 + b * v2 for a in [-size, size] for b in [-size, size]]
    x = np.array([p[0] for p in points]).reshape(2, 2)
    y = np.array([p[1] for p in points]).reshape(2, 2)
    z = np.array([p[2] for p in points]).reshape(2, 2)
    return x, y, z
def plot_vector_pair(u_vec, v_vec, scale=1.6, u_color='darkred', v_color='blue', name_prefix='principal'):
    return [
        go.Scatter3d(x=[0, u_vec[0]*scale], y=[0, u_vec[1]*scale], z=[0, u_vec[2]*scale],
                     mode='lines', line=dict(color=u_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} u"),
                     showlegend=False),
        go.Scatter3d(x=[0, v_vec[0]*scale], y=[0, v_vec[1]*scale], z=[0, v_vec[2]*scale],
                     mode='lines', line=dict(color=v_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} v"),
                     showlegend=False)
    ]
def angle_arc(u, v, origin=None, color='black', name_prefix="arc"):
    if origin is None:
        origin = np.zeros(3)

    u_norm = u / np.linalg.norm(u)
    v_norm = v / np.linalg.norm(v)

    angle = np.arccos(np.clip(np.dot(u_norm, v_norm), -1, 1))

    if np.isclose(angle, 0) or np.isclose(angle, np.pi):
        return None  # No arc needed

    # Build orthonormal basis u_norm, w in plane
    w = v_norm - np.dot(v_norm, u_norm) * u_norm
    if np.linalg.norm(w) < 1e-6:
        return None
    w /= np.linalg.norm(w)

    n_points = 30
    angles = np.linspace(0, angle, n_points)
    arc_points = []

    for θ in angles:
        p = np.cos(θ) * u_norm + np.sin(θ) * w
        arc_points.append(p + origin)

    arc_points = np.array(arc_points)

    return go.Scatter3d(
        x=arc_points[:, 0],
        y=arc_points[:, 1],
        z=arc_points[:, 2],
        mode='lines',
        line=dict(color=color, width=4, dash='dash'),
        showlegend=False,
        name=dynamic_name("arc", name_prefix)
    )
def plot_intersection_line(u1, u2, v1, v2, scale=2.5, name_prefix="Intersection Line"):
    nU = np.cross(u1, u2)
    nV = np.cross(v1, v2)
    line_dir = np.cross(nU, nV)

    if np.linalg.norm(line_dir) < 1e-6:
        return None  # No well-defined intersection

    line_dir /= np.linalg.norm(line_dir)

    return go.Scatter3d(
        x=[-scale*line_dir[0], scale*line_dir[0]],
        y=[-scale*line_dir[1], scale*line_dir[1]],
        z=[-scale*line_dir[2], scale*line_dir[2]],
        mode='lines',
        line=dict(color='black', width=3, dash='dash'),
        name=dynamic_name("intersection", name_prefix),
        showlegend=False  # Hide from legend to avoid jitter
    )

In [5]:
class PrincipalAnglePlayer(pn.viewable.Viewer):
    offset  = param.Number(default=0.0,  bounds=(-5.0, 5.0), label="Z Offset for Plane V")
    theta_1 = param.Integer(default=0,  bounds=(-1800, 1800), label="theta_1")  # 10*degrees
    theta_2 = param.Integer(default=90, bounds=(-1800, 1800), label="theta_2")  # 10*degrees

    r     = param.Number(default=3.5, bounds=(1.0, 10.0), label="Camera Distance")
    theta = param.Number(default=45,  bounds=(0, 360),    label="Azimuth (θ°)")
    phi   = param.Number(default=30,  bounds=(-90, 90),   label="Elevation (φ°)")

    def __init__(self, **params):
        super().__init__(**params)
        self.fig_pane = pn.pane.Plotly(height=650, config={'responsive': True})
        self.distances_pane = pn.pane.Markdown("", sizing_mode="stretch_width")

        self._locked_camera = None
        self.lock_button = pn.widgets.Button(name="Lock Current Camera", button_type="primary", width=200)
        self.lock_button.on_click(self._lock_camera_from_plot)

        self.param.watch(self._update_data_only, ['theta_1', 'theta_2', 'offset'])
        self.param.watch(self._update_camera_only, ['r', 'theta', 'phi'])

        self._current_camera = self._camera_eye()
        self._update_data_only()

    def _svd_from_planes(self, u1, u2, v1, v2):
        U, _ = np.linalg.qr(np.stack([u1, u2], axis=1))
        V, _ = np.linalg.qr(np.stack([v1, v2], axis=1))
        return np.linalg.svd(U.T @ V), U, V

    def compute_principal_vectors_and_angles(self, u1, u2, v1, v2):
        (W1, s, W2T), U, V = self._svd_from_planes(u1, u2, v1, v2)
        principal_U = U @ W1
        principal_V = V @ W2T.T
        angles = np.arccos(np.clip(s, -1, 1))
        return principal_U, principal_V, np.degrees(angles)

    def _camera_eye(self):
        θ, φ = np.radians(0.1*self.theta), np.radians(0.1*self.phi)
        zoom_factor = self.r
        x = zoom_factor * np.cos(θ) * np.cos(φ)
        y = zoom_factor * np.sin(θ) * np.cos(φ)
        z = zoom_factor * np.sin(φ)
        return dict(x=x, y=y, z=z)

    def _lock_camera_from_plot(self, _=None):
        if self.fig_pane.object and hasattr(self.fig_pane.object.layout.scene, "camera"):
            cam = self.fig_pane.object.layout.scene.camera.eye
            self._locked_camera = dict(x=cam.x, y=cam.y, z=cam.z)

    def _update_camera_only(self, *_):
        if self._locked_camera:
            self._current_camera = self._locked_camera
        else:
            self._current_camera = self._camera_eye()

        if self.fig_pane.object:
            self.fig_pane.object.update_layout(scene_camera=dict(eye=self._current_camera))

    def _update_data_only(self, *_):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])
        origin = np.zeros(3)

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        xU, yU, zU = create_plane(origin, u1, u2)
        xV, yV, zV = create_plane(origin + np.array([0, 0, self.offset]), v1, v2)

        principal_U, principal_V, angles = self.compute_principal_vectors_and_angles(u1, u2, v1, v2)

        # --- Create all new traces ---
        traces = [
            go.Surface(x=xU, y=yU, z=zU, opacity=0.85, showscale=False,
                       colorscale=[[0, '#989F7A'], [1, '#989F7A']],
                       name=dynamic_name("plane", "Plane U"), showlegend=False),
            go.Surface(x=xV, y=yV, z=zV, opacity=0.65, showscale=False,
                       colorscale=[[0, '#bbbbbb'], [1, '#bbbbbb']],
                       name=dynamic_name("plane", "Plane V"), showlegend=False)
        ]

        traces += plot_vector_pair(principal_U[:, 0], principal_V[:, 0], name_prefix='p1')
        traces += plot_vector_pair(principal_U[:, 1], principal_V[:, 1], name_prefix='p2')

        arc1 = angle_arc(principal_U[:, 0], principal_V[:, 0], origin=origin, color='black', name_prefix="θ₁ arc")
        arc2 = angle_arc(principal_U[:, 1], principal_V[:, 1], origin=origin, color='black', name_prefix="θ₂ arc")
        if arc1: traces.append(arc1)
        if arc2: traces.append(arc2)

        inter_line = plot_intersection_line(u1, u2, v1, v2)
        if inter_line: traces.append(inter_line)

        title_text = f"Principal Angles: θ₁ = {angles[0]:.2f}°, θ₂ = {angles[1]:.2f}°"
        # --- Full new figure (flicker-free), reusing locked or current camera ---
        camera_settings = dict(
            eye=self._locked_camera if self._locked_camera else self._camera_eye(),
            projection=dict(type="orthographic")
        )

        fig = go.Figure(data=traces)
        fig.update_layout(
            title=title_text,
            scene=dict(
                aspectmode='cube',
                xaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='X'),
                yaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='Y'),
                zaxis=dict(tickmode='array', tickvals=[-1, 0, 1, 2, 3], range=[-1.5, 3.0], title='Z'),
                camera=camera_settings
            ),
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12)),
            margin=dict(l=10, r=10, t=40, b=10),
            showlegend=True
        )

        self.fig_pane.object = fig
        self.distances_pane.object = self._compute_distances_markdown()

    def _compute_distances_markdown(self):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        (_, s, _), _, _ = self._svd_from_planes(u1, u2, v1, v2)
        s = np.clip(s, 0, 1)
        angles = np.arccos(s)

        spectral   = np.sin(np.max(angles))
        chordal    = np.sqrt(np.sum(np.sin(angles)**2))
        projection = np.sqrt(2) * chordal
        geodesic   = np.sqrt(np.sum(angles**2))

        text = f"""
        **Distances:**
        - **Spectral Distance:**   {spectral:20.4f}
        - **Chordal Distance:**    {chordal:20.4f}
        - **Projection Distance:** {projection:20.4f}
        - **Geodesic Distance:**   {geodesic:20.4f} radians
        """
        return text

    def __panel__(self):
        return pn.Row(
            pn.Column(
                pn.pane.Markdown("## Principal Angle Visualizer"),
                self.fig_pane,
                width=500,
            ),
            pn.Column(
                r"## Controls: θ₁ (Intersection), θ₂ (Inclination).",
                self.distances_pane,
                pn.Spacer(height=20),
                self.lock_button,
                pn.Spacer(height=20),

                "theta_1", pn.widgets.Player.from_param(self.param.theta_1, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                "theta_2", pn.widgets.Player.from_param(self.param.theta_2, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                width=500,
            ),
        )

In [6]:
from scipy.linalg import subspace_angles

def angle_to_vector(angle_deg, base_vector, orthogonal_direction):
    theta = np.radians(angle_deg)
    base = base_vector / np.linalg.norm(base_vector)
    ortho = orthogonal_direction / np.linalg.norm(orthogonal_direction)
    return np.cos(theta) * base + np.sin(theta) * ortho
def generate_subspaces(theta1_deg, theta2_deg):
    """
    Constructs a fixed 3D subspace V, and a 2D subspace U that varies with angles θ₁ and θ₂.
    """
    # Fixed 3D subspace V = span(e₁, e₂, e₅)
    v1 = np.array([1, 0, 0, 0, 0])  # e1
    v2 = np.array([0, 1, 0, 0, 0])  # e2
    v3 = np.array([0, 0, 0, 0, 1])  # e5
    V = np.stack([v1, v2, v3], axis=1)

    # Rotating 2D subspace U = span(u₁, u₂)
    e1 = np.array([1, 0, 0, 0, 0])
    e1_perp = np.array([0, 0, 1, 0, 0])  # rotates in (e₁, e₃)
    e2 = np.array([0, 1, 0, 0, 0])
    e2_perp = np.array([0, 0, 0, 1, 0])  # rotates in (e₂, e₄)

    u1 = angle_to_vector(theta1_deg, e1, e1_perp)
    u2 = angle_to_vector(theta2_deg, e2, e2_perp)
    U = np.stack([u1, u2], axis=1)

    return U, V
def projection_distance(U, V):
    """
    Standard projection distance: spectral norm of P_U - P_V
    May be dominated by static orthogonal components (e.g., v3 = e5).
    """
    PU = U @ U.T
    PV = V @ V.T
    return np.linalg.norm(PU - PV, ord=2)
def compute_distance_metrics(U, V):
    theta     = subspace_angles(U, V)
    sin_theta = np.sin(theta)

    return {
        'chordal':         np.linalg.norm(sin_theta),
        'spectral':        np.max(sin_theta),
        'projection':      projection_distance(U, V),
        'frobenius':       np.sqrt(2 * np.sum(sin_theta**2)),
        'geodesic':        np.linalg.norm(theta)
    }

In [7]:
class CombinedDistanceViewer(pn.viewable.Viewer):
    def __init__(self, resolution=91, **params):
        super().__init__(**params)

        self.θ_vals          = np.linspace(0, 90, resolution)
        self.metrics         = ['chordal', 'spectral', 'projection', 'geodesic', 'frobenius']
        self.metric_selector = pn.widgets.Select(name="Metric (Heatmap)", options=self.metrics, value='chordal')
        self.fix_toggle      = pn.widgets.RadioButtonGroup(name='Fix Angle', options=['θ₁ fixed', 'θ₂ fixed']) #, button_type='primary')
        self.angle_slider    = pn.widgets.FloatSlider(name="Fixed Angle (°)", start=0, end=90, step=1, value=30)

        self.data = self._precompute_data()

        self.plot_panel = pn.bind(self._make_plot, self.metric_selector, self.fix_toggle, self.angle_slider)

        self.layout = pn.Column(
            "## Combined Heatmap + Curve Viewer",
            pn.Row(self.metric_selector, self.fix_toggle, self.angle_slider),
            self.plot_panel
        )

    def _precompute_data(self):
        heatmaps = {m: np.zeros((len(self.θ_vals), len(self.θ_vals))) for m in self.metrics}
        for i, θ1 in enumerate(self.θ_vals):
            for j, θ2 in enumerate(self.θ_vals):
                U, V = generate_subspaces(θ1, θ2)
                d = compute_distance_metrics(U, V)
                for m in self.metrics:
                    heatmaps[m][j, i] = d[m]  # row=θ₂, col=θ₁
        return heatmaps

    def _make_plot(self, selected_metric, fix_label, fixed_angle):
        θ_vals = self.θ_vals
        idx = int(np.clip(np.round(fixed_angle), 0, 90))

        # HEATMAP: show selected metric
        Z = self.data[selected_metric].T
        heatmap = hv.Image((θ_vals, θ_vals, Z), kdims=['θ₁ (°)', 'θ₂ (°)'], vdims=[selected_metric]).opts(
            cmap='viridis', colorbar=True, width=400, height=400,
            title=f"{selected_metric.capitalize()} distance over (θ₁, θ₂)"
        )

        # SLICE line
        if fix_label == 'θ₁ fixed':
            line = hv.VLine(fixed_angle).opts(color='red', line_width=2)
        else:
            line = hv.HLine(fixed_angle).opts(color='red', line_width=2)

        heatmap_with_line = heatmap * line

        # CURVE: all 4 metrics across sweep
        curves = []
        for m in self.metrics:
            Zm = self.data[m]
            ys = Zm[:, idx] if fix_label == 'θ₁ fixed' else Zm[idx, :]
            x_axis = θ_vals
            xlabel = 'θ₂ (°)' if fix_label == 'θ₁ fixed' else 'θ₁ (°)'
            curve = hv.Curve((x_axis, ys), xlabel, "Distance", label=m).opts(ylim=(0,2.3))
            curves.append(curve)

        curve_overlay = hv.Overlay(curves).opts(
            #title=f"All Metrics vs. {'θ₂' if fix_label == 'θ₁ fixed' else 'θ₁'}",
            title=f"Metric Slices at Fixed {'θ₁' if fix_label == 'θ₁ fixed' else 'θ₂'} = {fixed_angle}°",
            legend_position='right',
            width=500,
            height=400
        )

        return pn.Row(heatmap_with_line, curve_overlay)

    def __panel__(self):
        return self.layout

#hv.extension('bokeh', logo=False)
#CombinedDistanceViewer().servable()

# 
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Computing Principal Angles</strong><br>
<strong style="height:100px;color:darkred;font-size:25px;">Algorithms and Examples</strong></div>

# 1. Review: Principal Angles Between Subspaces

**Previous notebooks in this series:**
* [**HyperplanesPrincipalAngles.ipynb**](HyperplanesPrincipalAngles.ipynb)
* [**HyperplaneDistances.ipynb**](HyperplaneDistances.ipynb)

**Principal Angles Between Subspaces**

* Let $\mathscr{C}(U), \mathscr{C}(V) \subset \mathbb{R}^n$ be subspaces of dimension $k$ and $\ell$, respectively.<br>
The **principal angles** $\theta_1 \leq \cdots \leq \theta_r$, where $r = \min(k, \ell)$,<br>
measure the smallest angles between directions in the two subspaces.

* If the columns of both $U$ and $V$ are mutually orthogonal and unit length,<br>
i.e., if they are **orthonormal basis matrices**,<br>
then the principal angles are determined by the **singular values of** $\mathbf{U^T V}$<br><br>
If $\{ \sigma_i, i=1,\dots,r \}\;$ are the singular values of $U^T V$, the principal angles are given by<br>
$\qquad \theta_i = \cos^{-1}(\sigma_i)$.

This formulation is the most practical and numerically stable for computation.

---
**Overview of This Notebook**

### 🧭 Focus of This Notebook

This notebook examines the **algorithmic and numerical aspects** of principal angle computation. We go beyond the core definition to explore:

- The importance of **orthonormalizing basis matrices**
- Comparing **QR and SVD orthonormalization** strategies
- Behavior under **near-degenerate or ill-conditioned subspaces**
- The effect of numerical precision on **subspace distance metrics**

Small-scale demos, reproducible examples, and visual diagnostics are included to highlight robustness (or failure) in practice.

# 2. Algorithmic Choices in Computing Principal Angles

To compute principal angles between two subspaces with bases $A, B \in \mathbb{R}^{n \times k}$,<br>
we must **orthonormalize** the columns.

A poor orthonormalization step can distort angles — especially the small ones<br>
as we have already seen in the previous notebook.

## 2.1 Orthonormalization

### 2.1.1 Option 1: QR Decomposition

Fast and effective for well-conditioned matrices: compute the QR decomposition of the matrices $A$ and $B$.<br>
This is fast and numerically stable.

* $\qquad$ Decompose $\;A = Q R\;$ and set $U = Q$.
* $\qquad$ Decompose $\;B = Q R\;$ and set $V = Q$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">
<!-- ----------------------------------------- Python box -->
<div style="width: 22em; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _ = np.linalg.qr(A)</div>
    <div>V, _ = np.linalg.qr(B)</div>
  </div>
</div>
<!-- ---------------------------------------- Julia box -->
<div style="width: 22em; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U = qr(A).Q</div>
    <div>V = qr(B).Q</div>
  </div>
</div>
</div>

### 2.1.2 Option 2: SVD-Based Orthonormalization

In this approach, we compute the **singular value decomposition** of the input matrices $A$ and $B$, and extract the left singular vectors to obtain orthonormal bases for their column spaces.

This method is **more robust** than QR when $A$ or $B$ are nearly rank-deficient or ill-conditioned. It is especially useful when numerical precision matters in detecting small principal angles.

* $\qquad$ Decompose $\;A = U \Sigma V^\top\;$ and set $U_{\text{orth}} = U$.
* $\qquad$ Decompose $\;B = U \Sigma V^\top\;$ and set $V_{\text{orth}} = U$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">

<!-- Python box -->
<div style="width: 10cm; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _, _ = np.linalg.svd(A, full_matrices=False)</div>
    <div>V, _, _ = np.linalg.svd(B, full_matrices=False)</div>
  </div>
</div>

<!-- Julia box -->
<div style="width: 8cm; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U, _, _ = svd(A, full=false)</div>
    <div>V, _, _ = svd(B, full=false)</div>
  </div>
</div>

</div>

### 2.1.3 Educational Example: QR Fails, SVD Succeeds

To highlight the **numerical limitations** of QR-based orthonormalization,<br>
$\qquad$ we construct a matrix $A \in \mathbb{R}^{n \times 2}$ whose columns are nearly linearly dependent<br>
$\qquad$ i.e., $A$ is **numerically rank-deficient**.

In this situation, the QR decomposition still produces two orthonormal vectors (because it assumes $A$ has full rank)<br>
$\qquad$ but the resulting basis **does not represent the true column space of $A$**. Specifically:

- The **first column** of $Q$ aligns with the true subspace (as expected),
- But the **second column** is orthogonal to it and **numerically meaningless**.

By contrast, the SVD identifies that $A$ is rank-1 and produces a **single orthonormal vector**<br>
$\qquad$ that spans the correct subspace.

____
We demonstrate this by:
- Comparing the **angle** between the dominant singular vector and each column of $Q$,
- Showing that QR returns a basis that is orthonormal, but **geometrically incorrect** in rank-deficient settings.


In [8]:
%%julia
Random.seed!(256)

# Construct a numerically rank-1 matrix A
n = 10
A = randn(n, 1)
A = [A  A .+ 1e-15 * randn(n)]   # Second column is nearly the same

println("Condition number C(A): ", cond(A))
println("Numerical     rank(A): ", rank(A))

# --- SVD: true subspace (rank-1)
U_svd = svd(A, full=false).U[:, 1]  # dominant left singular vector

# --- QR: full-rank orthonormal basis (but not meaningful)
Q_qr = Matrix(qr(A).Q)

# Check orthonormality
orth_qr = norm(Q_qr' * Q_qr - I, Inf)

# Compare Q_qr[:,1] with U_svd: should be aligned
angle_1 = acos(clamp(dot(U_svd, Q_qr[:,1]), -1, 1)) * 180 / π

# Compare Q_qr[:,2] with U_svd: should be ~90°
angle_2 = acos(clamp(dot(U_svd, Q_qr[:,2]), -1, 1)) * 180 / π

println("\nOrthonormality of QR basis: ‖QᵀQ - I‖∞ = ", orth_qr)
println("\nAngle between dominant SVD vector and QR vectors:")
println("  θ(Q[:,1], U_svd): ", round(angle_1, digits=6), " degrees")
println("  θ(Q[:,2], U_svd): ", round(angle_2, digits=6), " degrees")
;

Condition number C(A): 4.118150102172615e15
Numerical     rank(A): 1

Orthonormality of QR basis: ‖QᵀQ - I‖∞ = 1.1102230246251565e-16

Angle between dominant SVD vector and QR vectors:
  θ(Q[:,1], U_svd): 0.0 degrees
  θ(Q[:,2], U_svd): 90.0 degrees

In [120]:
class QRvsSVDDisplay(pn.viewable.Viewer, pn.custom.JSComponent):
    eps = param.Number(default=1e-4, bounds=(0.0, 0.05), doc="Perturbation ε")

    a1 = param.List(default=[1.0, 0.0])
    a2 = param.List(default=[0.0, 1.0])
    q1 = param.List(default=[1.0, 0.0])
    q2 = param.List(default=[0.0, 1.0])
    u1 = param.List(default=[1.0, 0.0])
    u2 = param.List(default=[0.0, 1.0])
    angle = param.Number(default=0.0)
    rank = param.Integer(default=2)

    _esm = """
    export function render({ model, el }) {
        const width = 400;
        const height = 400;
        const xrange = [-2.5, 2.5];
        const yrange = [-2.5, 2.5];

        const canvas = document.createElement('canvas');
        canvas.width = width;
        canvas.height = height;
        canvas.style.border = "1px solid black";
        el.appendChild(canvas);

        const paperScope = new paper.PaperScope();
        paperScope.setup(canvas);

        const worldToScreen = (point) => {
            const x = (point.x - xrange[0]) / (xrange[1] - xrange[0]) * width;
            const y = height - (point.y - yrange[0]) / (yrange[1] - yrange[0]) * height;
            return { x, y };
        };

        function drawArrow(start, end, color, label, dashed = false, labelOffset = null) {
            const startPoint = worldToScreen(start);
            const endPoint = worldToScreen(end);

            const line = new paperScope.Path.Line({
                from: new paperScope.Point(startPoint.x, startPoint.y),
                to: new paperScope.Point(endPoint.x, endPoint.y),
                strokeColor: color,
                strokeWidth: 2,
                dashArray: dashed ? [5, 5] : null,
            });

            const arrowSize = 10;
            const direction = new paperScope.Point(endPoint.x - startPoint.x, endPoint.y - startPoint.y).normalize();

            const arrowHead = new paperScope.Path({
                segments: [
                    new paperScope.Point(endPoint.x, endPoint.y),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize + direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize - direction.x * arrowSize / 2
                    ),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize - direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize + direction.x * arrowSize / 2
                    )
                ],
                closed: true,
                fillColor: color,
            });

            if (label) {
                let labelPoint = new paperScope.Point(endPoint.x, endPoint.y);
                if (labelOffset) {
                    labelPoint = labelPoint.add(new paperScope.Point(labelOffset.x, labelOffset.y));
                } else {
                    const perp = new paperScope.Point(-direction.y, direction.x).normalize().multiply(15);
                    labelPoint = labelPoint.add(perp);
                }
                new paperScope.PointText({
                    point: labelPoint,
                    content: label,
                    fillColor: color,
                    fontSize: 12,
                    justification: 'center',
                });
            }
        }

        function redraw() {
            paperScope.project.clear();

            const origin = { x: 0, y: 0 };

            const a1 = model.a1;
            const a2 = model.a2;
            const q1 = model.q1;
            const q2 = model.q2;
            const u1 = model.u1;
            const u2 = model.u2;

            drawArrow(origin, { x: a1[0], y: a1[1] }, 'gray', 'a₁', true, { x: -15, y: -5 });
            drawArrow(origin, { x: a2[0], y: a2[1] }, 'gray', 'a₂', true, { x: 15, y: 10 });
            drawArrow(origin, { x: q1[0], y: q1[1] }, 'blue', 'q₁', false, { x: -12, y: -10 });
            drawArrow(origin, { x: q2[0], y: q2[1] }, 'blue', 'q₂', false, { x: -12, y: 12 });
            drawArrow(origin, { x: u1[0], y: u1[1] }, 'red', 's₁', false, { x: 12, y: 10 });
            drawArrow(origin, { x: u2[0], y: u2[1] }, 'red', 's₂', false, { x: 12, y: -12 });
        }

        redraw();

        model.on(['a1', 'a2', 'q1', 'q2', 'u1', 'u2', 'angle', 'rank'], () => {
            //console.log("[DEBUG] model changed → redraw()");
            redraw();
        });

        paperScope.view.draw();
    }
    """

    def __init__(self, **params):
        super().__init__(**params)

        self._base_vector         = np.random.randn(2)
        self._perturb_direction   = np.random.randn(2)
        self._perturb_direction  /= np.linalg.norm(self._perturb_direction)

        self.eps_slider = pn.widgets.FloatSlider(
            name="ε (perturbation)", start=0.0, end=0.05, step=1e-4,
            format="0.0000", value=self.eps, width=300
        )
        self.eps_slider.param.watch(self._on_eps_change, 'value')

        self.perturb_a1 = pn.widgets.Checkbox(name="Perturb a₁ instead of a₂", value=False)
        self.perturb_a1.param.watch(self._on_mode_change, "value")

        self.regen_button = pn.widgets.Button(name="Reset A") #, button_type="primary")
        self.regen_button.on_click(self._on_regenerate)
        self.mode_selector = pn.widgets.Select(
            name="Mode",
            options=["Random A", "A Preset"],
            value="Random A",
            width=100,
        )
        self.mode_selector.param.watch(self._on_mode_change, "value")


        self.status = pn.pane.Markdown("", width=300)

        self._update_vectors()

        self.explanation = pn.pane.Markdown("""
<span style="font-size:15px;">

### What to Observe

- As **ε** increases, vectors **a₁** and **a₂** diverge.
- **QR** and **SVD* bases (**q₁**, **s₁**) rotate differently.<br>
  **∠(q_1, s_1)** captures this divergence.
- SVD typically remains **geometrically stable**,<br> while QR may drift.
- <span style="color:darkred;"><strong>Watch for θ ≥ 1° instability warning</strong></span>
</span>
""", width=400)


        self.layout = pn.Row(
            pn.Column(
                pn.pane.Markdown("### QR vs. SVD Orthonormalization"),
                self.eps_slider,
                pn.Row(
                    pn.Column(pn.Spacer(height=0),  self.mode_selector),
                    pn.Column(pn.Spacer(height=18), self.regen_button),
                    pn.Spacer(width=35),
                    pn.Column(pn.Spacer(height=26), self.perturb_a1),
                ),
                self,
            ),
            pn.Spacer(width=40),
            pn.Column(
                pn.Spacer(height=120),
                self.status,
                pn.Spacer(height=30),
                self.explanation,
            )
        )

    def _on_eps_change(self, event):
        self.eps = event.new
        self._update_vectors()

    def _on_mode_change(self, event):
        self._on_regenerate(None)

    def _on_regenerate(self, event):
        if self.mode_selector.value == "Hard-coded example":
            self._base_vector        = [0.69845715, 0.00377089]
            self._perturb_direction  = np.array([0.93943317, 0.34273214])
        else:
            self._base_vector       = np.random.randn(2)
            self._perturb_direction = np.random.randn(2)
            self._perturb_direction /= np.linalg.norm(self._perturb_direction)

        self._update_vectors()

    def _update_vectors(self):
        eps = self.eps
        a1 = self._base_vector.copy()

        if self.perturb_a1.value:
            a1 = a1 + eps * self._perturb_direction
            a2 = self._base_vector
        else:
            a2 = a1 + eps * self._perturb_direction

        A = np.column_stack((a1, a2))
        Q, _ = qr(A)
        U, _, _ = svd(A, full_matrices=False)

        cosθ      = np.clip(np.dot(Q[:, 0], U[:, 0]), -1.0, 1.0)
        angle_deg = float(np.degrees(np.arccos(cosθ)))
        rank      = int(np.linalg.matrix_rank(A))
        diff      = np.linalg.norm(a2 - a1)
        angle_style = "color:red;" if angle_deg >= 1.0 else ""

        #print("[DEBUG] _update_vectors called")
        #print(f"[DEBUG] eps = {eps}")
        #print(f"[DEBUG] a1 = {a1}")
        #print(f"[DEBUG] a2 = {a2}")
        #print(f"[DEBUG] angle = {angle_deg}°, rank = {rank}")
        #print(f"[DEBUG] diff = {diff}")

        self.param.update(
            a1=a1.tolist(),
            a2=a2.tolist(),
            q1=Q[:, 0].tolist(),
            q2=Q[:, 1].tolist(),
            u1=U[:, 0].tolist(),
            u2=U[:, 1].tolist(),
            angle=angle_deg,
            rank=rank
        )

        self.status.object = f"""
<table style="font-size:14px; border-collapse:collapse; border:none;">
  <tr>
    <td style="padding:2px 8px; border:none;"><b>ε</b></td>
    <td style="text-align:right; padding:2px 8px; border:none; font-family:monospace; min-width:80px;">{eps:.4g}</td>
  </tr>
  <tr>
    <td style="padding:2px 8px; border:none;"><b>‖a₂ − a₁‖</b></td>
    <td style="text-align:right; padding:2px 8px; border:none; font-family:monospace; min-width:80px;">{diff:.4g}</td>
  </tr>
  <tr>
    <td style="padding:2px 8px; border:none;"><b>∠(q₁, s₁)</b></td>
    <td style="text-align:right; padding:2px 8px; border:none; font-family:monospace; min-width:80px; {angle_style}">{angle_deg:.4f}°</td>
  </tr>
  <tr>
    <td style="padding:2px 8px; border:none;"><b>rank(A)</b></td>
    <td style="text-align:right; padding:2px 8px; border:none; font-family:monospace; min-width:80px;">{rank}</td>
  </tr>
</table>
"""

    def __panel__(self):
        return self.layout

qr_vs_svd = QRvsSVDDisplay()
qr_vs_svd.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] FloatSlider(end=0.05, format='0.0000', name='ε (perturbation)', step=0.0001, value=0.0001, width=300)
        [2] Row
            [0] Column
                [0] Spacer(height=0)
                [1] Select(name='Mode', options=['Random A', 'A Preset'], value='Random A', width=100)
            [1] Column
                [0] Spacer(height=18)
                [1] Button(name='Reset A')
            [2] Spacer(width=35)
            [3] Column
                [0] Spacer(height=26)
                [1] Checkbox(name='Perturb a₁ instead o...)
        [3] QRvsSVDDisplay(a1=[-0.3091144447717088, ...], a2=[-0.30902143900923856, ...], angle=0.0019771676017550045, q1=[-0.7979746819370039, ...], q2=[-0.6026909713837909, ...], u1=[-0.7979538837519908, ...], u2=[-0.6027185076012799, ...])
    [1] Spacer(width=40)
    [2] Column
        [0] Spacer(height=120)
        [1] Markdown(str, width=300)
        [2] Spacer(height=30)
        [3] Markdown(str, width=400)

In [125]:

class VectorAngleViewer(pn.viewable.Viewer, pn.custom.JSComponent):
    angle = param.Number(default=45, bounds=(0, 180), label="Angle (degrees)")

    _esm = """
    export function render({ model, el }) {
        const width  = 300;
        const height = 270;
        const xrange = [-1.5, 1.5];
        const yrange = [-0.3, 2.0];

        const canvas = document.createElement('canvas');
        canvas.width = width;
        canvas.height = height;
        canvas.style.border = "1px solid black";
        el.appendChild(canvas);

        const paperScope = new paper.PaperScope();
        paperScope.setup(canvas);

        const worldToScreen = (point) => {
            const x = (point.x - xrange[0]) / (xrange[1] - xrange[0]) * width;
            const y = height - (point.y - yrange[0]) / (yrange[1] - yrange[0]) * height;
            return new paperScope.Point(x, y);
        };

        function drawArrow(start, end, color, label, dashed = false, labelOffset = null) {
            const startPoint = worldToScreen(start);
            const endPoint = worldToScreen(end);

            const line = new paperScope.Path.Line({
                from: new paperScope.Point(startPoint.x, startPoint.y),
                to: new paperScope.Point(endPoint.x, endPoint.y),
                strokeColor: color,
                strokeWidth: 2,
                dashArray: dashed ? [5, 5] : null,
            });

            const arrowSize = 10;
            const direction = new paperScope.Point(endPoint.x - startPoint.x, endPoint.y - startPoint.y).normalize();

            const arrowHead = new paperScope.Path({
                segments: [
                    new paperScope.Point(endPoint.x, endPoint.y),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize + direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize - direction.x * arrowSize / 2
                    ),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize - direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize + direction.x * arrowSize / 2
                    )
                ],
                closed: true,
                fillColor: color,
            });

            if (label) {
                let labelPoint = new paperScope.Point(endPoint.x, endPoint.y);
                if (labelOffset) {
                    labelPoint = labelPoint.add(new paperScope.Point(labelOffset.x, labelOffset.y));
                } else {
                    const perp = new paperScope.Point(-direction.y, direction.x).normalize().multiply(15);
                    labelPoint = labelPoint.add(perp);
                }
                new paperScope.PointText({
                    point: labelPoint,
                    content: label,
                    fillColor: color,
                    fontSize: 12,
                    justification: 'center',
                });
            }
        }

        function redraw() {
            paperScope.project.clear();

            const origin = { x: 0, y: 0 };
            const angleRad = model.angle * Math.PI / 180;
            const u = { x: 1, y: 0 };
            const v = { x: Math.cos(angleRad), y: Math.sin(angleRad) };

            // Draw arrows for u and v
            drawArrow(origin, u, 'blue', 'u');
            drawArrow(origin, v, 'blue', 'v');

            // Draw arc
            const arcRadius = 0.3;
            const arcCenter = worldToScreen(origin);
            const midAngle = angleRad / 2;
            const mid = {
                x: arcRadius * Math.cos(midAngle),
                y: arcRadius * Math.sin(midAngle)
            };

            const arc = new paperScope.Path.Arc({
                from: worldToScreen({ x: arcRadius, y: 0 }),
                through: worldToScreen(mid),
                to: worldToScreen({ x: arcRadius * Math.cos(angleRad), y: arcRadius * Math.sin(angleRad) }),
                strokeColor: 'blue',
                strokeWidth: 2
            });

            // Add angle label
            const labelOffset = { x: 10, y: -10 };
            drawArrow(origin, v, 'blue', null);  // duplicate so label offset works
            const labelPoint = worldToScreen(mid);
            new paperScope.PointText({
                point: labelPoint.add(new paperScope.Point(labelOffset.x, labelOffset.y)),
                content: `${model.angle.toFixed(1)}°`,
                fillColor: 'black',
                fontSize: 14,
                justification: 'center'
            });
        }

        redraw();
        model.on("angle", () => redraw());
    }
    """

    def __init__(self, **params):
        super().__init__(**params)
        self.latex_pane = pn.pane.LaTeX(width=300, renderer='katex')
        self.slider = pn.widgets.FloatSlider.from_param(self.param.angle, width=300)
        self.param.watch(self._update_latex, 'angle')
        self._update_latex()

    def _update_latex(self, *_):
        angle_rad = np.radians(self.angle)
        u = np.array([1.0, 0.0])
        v = np.array([np.cos(angle_rad), np.sin(angle_rad)])
        dot_product = np.dot(u, v)
        self.latex_pane.object = (
            "<div style='font-size:16px;font-family:monospace;'>"
            r"$\Large \begin{aligned}"
            r"u &= \begin{bmatrix} 1.00 \\ 0.00 \end{bmatrix} \\ "
            rf"v &= \begin{{bmatrix}} {v[0]:.2f} \\ {v[1]:.2f} \end{{bmatrix}} \\ "
            rf"u \cdot v &= {dot_product:.2f} \\ "
            rf"\theta &= {self.angle:.2f}^\circ"
            r"\end{aligned}$"
            "</span>"
        )

    def __panel__(self):
        return pn.Row(
            self,
            pn.Spacer(width=30),
            pn.Column(
                pn.Spacer(height=20),
                self.latex_pane,
                pn.Spacer(height=20),
                self.slider,
            )
        )


VectorAngleViewer()

VectorAngleViewer()